In [7]:
'''
Visualization of the filters of a CNN, via gradient ascent in input space.
This script can run on CPU in a few minutes.
This script is meant for use only and is not discussed in great details as it is beyond the scope of the class.
'''
# using tf.2.1 in colab
# %tensorflow_version 2.x

# from __future__ import print_function

import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.preprocessing.image import save_img
from tensorflow.keras import backend as K
# from google.colab import drive
# stopping eager execution mode to be allowed to use the gradient function
tf.compat.v1.disable_eager_execution()
# sanity check for tf version
print(tf.__version__)

###################################################################################################
# dimensions of the generated pictures for each filter.
###################################################################################################   
img_width = 150
img_height = 150

###################################################################################################
# the name of the layer we want to visualize 
###################################################################################################   
layer_name='conv2d_15'

###################################################################################################
# util function to convert a tensor into a valid image
###################################################################################################   
def deprocess_image(x):
    
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

###################################################################################################
# Load the model
###################################################################################################   
from tensorflow.keras.models import load_model
model=load_model('./dogs_cats_birds_model_case01_30epoch')

###################################################################################################
# This is the placeholder for the input images
###################################################################################################   
input_img = model.input

###################################################################################################
# Get the symbolic outputs of each "key" layer (we gave them unique names).
###################################################################################################   
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

###################################################################################################
# Utility function to normalize a tensor by its L2 norm
###################################################################################################   
def normalize(x): 
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

kept_filters = []

###################################################################################################
# Scan through some number of filters...
###################################################################################################   

for filter_index in range(32):

    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(100):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)

    # decode the resulting input image
    if True:
        
    #if loss_value > 0
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

###################################################################################################
# we will stich the best n^2 filters on a n x n grid.
###################################################################################################
n = 5

###################################################################################################
# the filters that have the highest loss are assumed to be more intuitive
# we will only keep the top n filters.
###################################################################################################   
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

###################################################################################################
# build a black picture with enough space for
# our n x n filters of size 128 x 128, with a 5px margin in between
###################################################################################################   
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

###################################################################################################
# fill the picture with our saved filters 
###################################################################################################   
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

###################################################################################################
# save the result to drive
###################################################################################################   
save_img('./dogs_cats_birds_model_case01_30epoch_%dx%d.png' % (n, n), stitched_filters)
print('saved')

2.1.0
Processing filter 0
Current loss value: 58.190826
Current loss value: 67.05486
Current loss value: 73.537735
Current loss value: 80.539734
Current loss value: 88.43503
Current loss value: 95.440094
Current loss value: 103.57461
Current loss value: 112.436615
Current loss value: 131.22353
Current loss value: 146.37094
Current loss value: 156.43237
Current loss value: 174.6813
Current loss value: 190.09659
Current loss value: 215.79419
Current loss value: 240.87492
Current loss value: 275.1112
Current loss value: 307.70905
Current loss value: 326.18933
Current loss value: 356.52634
Current loss value: 387.6999
Current loss value: 413.53946
Current loss value: 451.17963
Current loss value: 478.3758
Current loss value: 516.361
Current loss value: 556.5848
Current loss value: 599.4784
Current loss value: 642.07385
Current loss value: 687.874
Current loss value: 741.00104
Current loss value: 784.4508
Current loss value: 829.0562
Current loss value: 876.62317
Current loss value: 930.573

Current loss value: -36.33091
Current loss value: -33.728287
Current loss value: -30.72257
Current loss value: -28.072084
Current loss value: -25.616539
Current loss value: -24.718224
Current loss value: -23.355621
Current loss value: -22.211292
Current loss value: -21.568428
Current loss value: -21.063293
Current loss value: -20.22311
Current loss value: -19.262295
Current loss value: -19.376144
Current loss value: -19.227695
Current loss value: -18.461872
Current loss value: -17.407473
Current loss value: -18.639036
Current loss value: -16.610445
Current loss value: -16.41098
Current loss value: -17.126274
Current loss value: -15.112036
Current loss value: -16.244028
Current loss value: -15.4270115
Current loss value: -18.189587
Current loss value: -14.671915
Current loss value: -13.7725525
Current loss value: -17.532349
Current loss value: -13.658154
Current loss value: -14.634012
Current loss value: -14.132191
Current loss value: -14.327047
Current loss value: -13.480942
Current lo

Current loss value: 751.8788
Current loss value: 766.5249
Current loss value: 778.06134
Current loss value: 793.04517
Current loss value: 804.6802
Current loss value: 819.9715
Current loss value: 833.9236
Current loss value: 846.897
Current loss value: 861.3718
Current loss value: 874.63055
Current loss value: 888.65704
Current loss value: 903.55945
Current loss value: 917.3801
Current loss value: 929.3968
Current loss value: 943.0892
Current loss value: 956.9847
Current loss value: 972.7296
Current loss value: 986.8293
Current loss value: 998.1062
Current loss value: 1013.6114
Current loss value: 1027.5547
Current loss value: 1043.0393
Current loss value: 1056.9412
Current loss value: 1070.0886
Current loss value: 1081.9586
Filter 5 processed in 0s
Processing filter 6
Current loss value: -52.511818
Current loss value: -50.211914
Current loss value: -45.445976
Current loss value: -41.99386
Current loss value: -35.56675
Current loss value: -30.228678
Current loss value: -27.03701
Curren

Current loss value: -8.041393
Current loss value: -8.1327915
Current loss value: -6.092209
Current loss value: -8.514969
Current loss value: -5.7385798
Current loss value: -7.668219
Current loss value: -6.1145067
Current loss value: -3.7770844
Current loss value: -4.4806986
Current loss value: -3.444238
Current loss value: -5.262323
Current loss value: -2.2856376
Current loss value: -2.1433415
Current loss value: -2.234129
Current loss value: -2.8942416
Current loss value: -1.7185553
Current loss value: -0.30721688
Current loss value: -0.8852604
Current loss value: 0.8586358
Current loss value: 0.70041645
Current loss value: 1.3131647
Current loss value: 1.7960832
Current loss value: 2.1689348
Current loss value: 2.1519918
Current loss value: 2.3634572
Current loss value: 3.3570912
Current loss value: 3.673658
Current loss value: 3.7729077
Current loss value: 3.1903284
Current loss value: 4.2477894
Current loss value: 4.6906776
Current loss value: 5.8395286
Filter 8 processed in 0s
Pro

Current loss value: 696.66504
Current loss value: 713.9215
Current loss value: 735.23584
Current loss value: 754.43164
Current loss value: 773.5559
Current loss value: 794.0565
Current loss value: 813.26587
Current loss value: 833.5437
Current loss value: 851.0097
Current loss value: 871.808
Current loss value: 891.46844
Current loss value: 910.6299
Current loss value: 929.4684
Current loss value: 950.2528
Current loss value: 969.8609
Current loss value: 988.02625
Current loss value: 1007.60486
Current loss value: 1027.0905
Current loss value: 1046.7091
Current loss value: 1064.093
Current loss value: 1084.3953
Current loss value: 1104.2068
Current loss value: 1125.1653
Current loss value: 1144.0981
Current loss value: 1164.4338
Current loss value: 1184.3862
Current loss value: 1203.3619
Current loss value: 1222.7557
Current loss value: 1242.3733
Current loss value: 1261.0042
Current loss value: 1281.2227
Current loss value: 1301.2533
Current loss value: 1319.7493
Filter 11 processed i

...

Current loss value: 25145.475
Current loss value: 25558.012
Current loss value: 25969.416
Current loss value: 26377.547
Current loss value: 26794.637
Current loss value: 27199.12
Current loss value: 27608.914
Current loss value: 28022.629
Current loss value: 28428.84
Current loss value: 28835.703
Current loss value: 29249.271
Current loss value: 29658.275
Current loss value: 30070.225
Current loss value: 30483.258
Current loss value: 30894.914
Current loss value: 31304.275
Current loss value: 31716.164
Current loss value: 32122.4
Current loss value: 32540.55
Current loss value: 32951.62
Current loss value: 33363.89
Current loss value: 33769.848
Current loss value: 34175.305
Current loss value: 34578.73
Current loss value: 34984.53
Current loss value: 35390.676
Current loss value: 35798.82
Current loss value: 36210.727
Current loss value: 36617.234
Current loss value: 37020.258
Current loss value: 37431.348
Current loss value: 37837.676
Filter 17 processed in 1s
Processing filter 18
Cur

Current loss value: -32.73864
Current loss value: -29.22454
Current loss value: -28.98596
Current loss value: -27.005827
Current loss value: -25.377792
Current loss value: -23.744032
Current loss value: -23.704882
Current loss value: -22.614601
Current loss value: -19.602697
Current loss value: -19.149622
Current loss value: -18.145119
Current loss value: -18.567484
Current loss value: -18.025192
Current loss value: -15.966797
Current loss value: -13.931558
Current loss value: -14.015034
Current loss value: -11.17058
Current loss value: -11.33242
Current loss value: -8.1260805
Current loss value: -8.962112
Current loss value: -5.2708344
Current loss value: -2.2490804
Current loss value: 2.4810486
Current loss value: 5.8948784
Current loss value: 13.553533
Current loss value: 23.47378
Current loss value: 33.131866
Current loss value: 42.617405
Current loss value: 51.790604
Current loss value: 65.37114
Current loss value: 77.433304
Current loss value: 91.01999
Current loss value: 104.325

Current loss value: 16172.111
Current loss value: 16426.117
Current loss value: 16685.17
Current loss value: 16942.336
Current loss value: 17199.46
Current loss value: 17453.248
Current loss value: 17713.16
Current loss value: 17973.693
Current loss value: 18228.771
Current loss value: 18486.305
Current loss value: 18742.82
Current loss value: 19000.436
Current loss value: 19258.262
Current loss value: 19517.926
Current loss value: 19775.137
Current loss value: 20031.52
Current loss value: 20290.0
Current loss value: 20550.328
Current loss value: 20804.506
Current loss value: 21061.074
Current loss value: 21317.008
Current loss value: 21576.775
Current loss value: 21829.77
Current loss value: 22085.799
Current loss value: 22344.062
Current loss value: 22599.08
Current loss value: 22854.887
Current loss value: 23113.145
Current loss value: 23368.852
Current loss value: 23626.893
Filter 23 processed in 1s
Processing filter 24
Current loss value: -35.825897
Current loss value: -33.72107
C

Current loss value: 6406.3613
Current loss value: 6507.927
Current loss value: 6606.3247
Current loss value: 6707.8057
Current loss value: 6809.6743
Current loss value: 6911.4136
Current loss value: 7008.2627
Current loss value: 7113.362
Current loss value: 7214.4272
Current loss value: 7313.913
Current loss value: 7414.251
Current loss value: 7512.6045
Current loss value: 7618.0034
Current loss value: 7714.5083
Current loss value: 7821.5703
Current loss value: 7919.9805
Current loss value: 8022.537
Current loss value: 8127.412
Current loss value: 8230.395
Current loss value: 8336.372
Current loss value: 8439.902
Current loss value: 8543.93
Current loss value: 8648.027
Current loss value: 8750.429
Current loss value: 8854.231
Current loss value: 8956.891
Current loss value: 9056.156
Filter 26 processed in 1s
Processing filter 27
Current loss value: 58.804005
Current loss value: 79.58775
Current loss value: 110.39461
Current loss value: 152.03151
Current loss value: 205.55411
Current lo

Current loss value: 75.42316
Current loss value: 76.76503
Current loss value: 78.82529
Current loss value: 80.02954
Current loss value: 81.55475
Current loss value: 83.37727
Current loss value: 85.4971
Current loss value: 86.22229
Current loss value: 88.98266
Current loss value: 90.05167
Current loss value: 93.43274
Current loss value: 94.97279
Current loss value: 98.138596
Current loss value: 100.472015
Current loss value: 103.83599
Current loss value: 106.87897
Current loss value: 111.00333
Current loss value: 114.628845
Current loss value: 117.731834
Current loss value: 121.30548
Current loss value: 124.938
Current loss value: 129.49445
Current loss value: 133.07703
Current loss value: 136.42422
Current loss value: 140.67104
Current loss value: 144.47131
Current loss value: 148.13599
Current loss value: 152.51094
Current loss value: 156.57004
Current loss value: 161.25586
Current loss value: 164.07156
Current loss value: 168.60724
Current loss value: 173.1994
Filter 29 processed in 